<a href="https://colab.research.google.com/github/ArunKrishnan09/Covid-Analysis/blob/main/Covid_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from io import StringIO
import numpy as np
import sqlite3

In [145]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
import os
os.chdir("/content/drive/MyDrive/Arun-ads1/Covid project")

In [147]:
misc_data = pd.read_csv("OWID_COVID_Data.csv")


In [148]:
misc_data = misc_data[["iso_code", "location", "date", "population",
    'population_density', 'median_age', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'life_expectancy', 'human_development_index']]

In [149]:
urls = ["https://catalog.ourworldindata.org/garden/covid/latest/cases_deaths/cases_deaths.csv", "https://catalog.ourworldindata.org/garden/covid/latest/hospital/hospital.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_global/vaccinations_global.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_age/vaccinations_age.csv", "https://catalog.ourworldindata.org/garden/covid/latest/testing/testing.csv", "https://catalog.ourworldindata.org/garden/covid/latest/google_mobility/google_mobility.csv", "https://catalog.ourworldindata.org/garden/covid/latest/oxcgrt_policy/oxcgrt_policy.csv"]

In [150]:
'''
CODE FOR DOWNLOADING DATASETS
import requests

urls = ["https://catalog.ourworldindata.org/garden/covid/latest/cases_deaths/cases_deaths.csv", "https://catalog.ourworldindata.org/garden/covid/latest/hospital/hospital.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_global/vaccinations_global.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_age/vaccinations_age.csv", "https://catalog.ourworldindata.org/garden/covid/latest/testing/testing.csv", "https://catalog.ourworldindata.org/garden/covid/latest/google_mobility/google_mobility.csv", "https://catalog.ourworldindata.org/garden/covid/latest/oxcgrt_policy/oxcgrt_policy.csv"]
dataset_names = ["cases_deaths", "hospital", "vaccinations_global", "vaccinations_age", "testing", "google_mobility", "oxcgrt_policy"]

for i, url in enumerate(urls):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        df = pd.read_csv(StringIO(response.text))
        df.to_csv(f"{dataset_names[i]}.csv", index=False)
        print(f"Downloaded and saved {dataset_names[i]}.csv")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
    except Exception as e:
        print(f"Error processing {url}: {e}")
'''

'\nCODE FOR DOWNLOADING DATASETS\nimport requests\n\nurls = ["https://catalog.ourworldindata.org/garden/covid/latest/cases_deaths/cases_deaths.csv", "https://catalog.ourworldindata.org/garden/covid/latest/hospital/hospital.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_global/vaccinations_global.csv", "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_age/vaccinations_age.csv", "https://catalog.ourworldindata.org/garden/covid/latest/testing/testing.csv", "https://catalog.ourworldindata.org/garden/covid/latest/google_mobility/google_mobility.csv", "https://catalog.ourworldindata.org/garden/covid/latest/oxcgrt_policy/oxcgrt_policy.csv"]\ndataset_names = ["cases_deaths", "hospital", "vaccinations_global", "vaccinations_age", "testing", "google_mobility", "oxcgrt_policy"]\n\nfor i, url in enumerate(urls):\n    try:\n        response = requests.get(url)\n        response.raise_for_status()  # Raise an exception for bad status codes\n\n        df

In [151]:
cases_deaths = pd.read_csv("cases_deaths.csv")
hospital = pd.read_csv("hospital.csv")
vaccinations_global = pd.read_csv("vaccinations_global.csv")
testing = pd.read_csv("testing.csv")
oxcgrt_policy = pd.read_csv("oxcgrt_policy.csv")

In [152]:
datasets = [cases_deaths, hospital, vaccinations_global, testing, oxcgrt_policy, misc_data]

In [153]:
for dataset in datasets:
  print(dataset.isnull().sum())

country                                         0
date                                            0
new_cases                                    2284
total_cases                                     0
new_deaths                                   1655
total_deaths                                    0
weekly_cases                                 3563
weekly_deaths                                2905
weekly_pct_growth_cases                    144072
weekly_pct_growth_deaths                   249307
biweekly_cases                               5367
biweekly_deaths                              4657
biweekly_pct_growth_cases                  131348
biweekly_pct_growth_deaths                 230392
new_cases_per_million                        2284
new_deaths_per_million                       1655
total_cases_per_million                         0
total_deaths_per_million                        0
weekly_cases_per_million                     3563
weekly_deaths_per_million                    2905


CLEANING CASE DATA

In [154]:
cases_deaths.drop(["total_deaths_per_million_last12m", "total_deaths_per_100k_last12m", "total_deaths_last12m", "biweekly_pct_growth_deaths", "biweekly_pct_growth_cases", "weekly_pct_growth_cases", "weekly_pct_growth_deaths", "cfr_short_term"], axis =1) #Dropping columns with tons of null (over 150k or so)

,country,date,new_cases,total_cases,new_deaths,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths,...,new_cases_per_million_7_day_avg_right,new_deaths_per_million_7_day_avg_right,new_deaths_per_100k_7_day_avg_right,cfr,cfr_100_cases,days_since_100_total_cases,days_since_5_total_deaths,days_since_1_total_cases_per_million,days_since_0_1_total_deaths_per_million,days_since_100_total_cases_and_5m_pop
0,Afghanistan,2020-01-04,0.0,0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-86.0,-91.0,-81.0,-91.0,-86.0
1,Afghanistan,2020-01-05,0.0,0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-85.0,-90.0,-80.0,-90.0,-85.0
2,Afghanistan,2020-01-06,0.0,0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-84.0,-89.0,-79.0,-89.0,-84.0
3,Afghanistan,2020-01-07,0.0,0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-83.0,-88.0,-78.0,-88.0,-83.0
4,Afghanistan,2020-01-08,0.0,0,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-82.0,-87.0,-77.0,-87.0,-82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455495,Zimbabwe,2024-12-25,0.0,266399,0.0,5740,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.154663,2.154663,1674.0,1756.0,1716.0,1756.0,1674.0
455496,Zimbabwe,2024-12-26,0.0,266399,0.0,5740,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.154663,2.154663,1675.0,1757.0,1717.0,1757.0,1675.0
455497,Zimbabwe,2024-12-27,0.0,266399,0.0,5740,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.154663,2.154663,1676.0,1758.0,1718.0,1758.0,1676.0
455498,Zimbabwe,2024-12-28,0.0,266399,0.0,5740,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.154663,2.154663,1677.0,1759.0,1719.0,1759.0,1677.0


In [155]:
cases_deaths.fillna({"new_cases" : 0, "new_deaths" :0, "weekly_cases" :0, "weekly_deaths" :0})
#the nulls are normally around the start of covid, where there would be very few new cases. Which led me to believe that its zero at these spots

,country,date,new_cases,total_cases,new_deaths,total_deaths,weekly_cases,weekly_deaths,weekly_pct_growth_cases,weekly_pct_growth_deaths,...,cfr_100_cases,cfr_short_term,days_since_100_total_cases,days_since_5_total_deaths,days_since_1_total_cases_per_million,days_since_0_1_total_deaths_per_million,days_since_100_total_cases_and_5m_pop,total_deaths_last12m,total_deaths_per_100k_last12m,total_deaths_per_million_last12m
0,Afghanistan,2020-01-04,0.0,0,0.0,0,0.0,0.0,NaN,NaN,...,NaN,NaN,-86.0,-91.0,-81.0,-91.0,-86.0,NaN,NaN,NaN
1,Afghanistan,2020-01-05,0.0,0,0.0,0,0.0,0.0,NaN,NaN,...,NaN,NaN,-85.0,-90.0,-80.0,-90.0,-85.0,NaN,NaN,NaN
2,Afghanistan,2020-01-06,0.0,0,0.0,0,0.0,0.0,NaN,NaN,...,NaN,NaN,-84.0,-89.0,-79.0,-89.0,-84.0,NaN,NaN,NaN
3,Afghanistan,2020-01-07,0.0,0,0.0,0,0.0,0.0,NaN,NaN,...,NaN,NaN,-83.0,-88.0,-78.0,-88.0,-83.0,NaN,NaN,NaN
4,Afghanistan,2020-01-08,0.0,0,0.0,0,0.0,0.0,NaN,NaN,...,NaN,NaN,-82.0,-87.0,-77.0,-87.0,-82.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455495,Zimbabwe,2024-12-25,0.0,266399,0.0,5740,0.0,0.0,NaN,NaN,...,2.154663,NaN,1674.0,1756.0,1716.0,1756.0,1674.0,3.0,0.018669,0.186694
455496,Zimbabwe,2024-12-26,0.0,266399,0.0,5740,0.0,0.0,NaN,NaN,...,2.154663,NaN,1675.0,1757.0,1717.0,1757.0,1675.0,3.0,0.018669,0.186694
455497,Zimbabwe,2024-12-27,0.0,266399,0.0,5740,0.0,0.0,NaN,NaN,...,2.154663,NaN,1676.0,1758.0,1718.0,1758.0,1676.0,3.0,0.018669,0.186694
455498,Zimbabwe,2024-12-28,0.0,266399,0.0,5740,0.0,0.0,NaN,NaN,...,2.154663,NaN,1677.0,1759.0,1719.0,1759.0,1677.0,3.0,0.018669,0.186694


In [156]:
values = {}
for col in cases_deaths.columns:
  if col != "country" and col != "date":
    values[col] = cases_deaths[col].median()
cases_deaths.fillna(value=values)

,country,date,new_cases,total_cases,new_deaths,total_deaths,weekly_cases,weekly_deaths,weekly_pct_growth_cases,weekly_pct_growth_deaths,...,cfr_100_cases,cfr_short_term,days_since_100_total_cases,days_since_5_total_deaths,days_since_1_total_cases_per_million,days_since_0_1_total_deaths_per_million,days_since_100_total_cases_and_5m_pop,total_deaths_last12m,total_deaths_per_100k_last12m,total_deaths_per_million_last12m
0,Afghanistan,2020-01-04,0.0,0,0.0,0,66.0,0.0,-3.3,-3.4,...,1.181207,0.391389,-86.0,-91.0,-81.0,-91.0,-86.0,0.0,0.000000,0.000000
1,Afghanistan,2020-01-05,0.0,0,0.0,0,66.0,0.0,-3.3,-3.4,...,1.181207,0.391389,-85.0,-90.0,-80.0,-90.0,-85.0,0.0,0.000000,0.000000
2,Afghanistan,2020-01-06,0.0,0,0.0,0,66.0,0.0,-3.3,-3.4,...,1.181207,0.391389,-84.0,-89.0,-79.0,-89.0,-84.0,0.0,0.000000,0.000000
3,Afghanistan,2020-01-07,0.0,0,0.0,0,66.0,0.0,-3.3,-3.4,...,1.181207,0.391389,-83.0,-88.0,-78.0,-88.0,-83.0,0.0,0.000000,0.000000
4,Afghanistan,2020-01-08,0.0,0,0.0,0,66.0,0.0,-3.3,-3.4,...,1.181207,0.391389,-82.0,-87.0,-77.0,-87.0,-82.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455495,Zimbabwe,2024-12-25,0.0,266399,0.0,5740,0.0,0.0,-3.3,-3.4,...,2.154663,0.391389,1674.0,1756.0,1716.0,1756.0,1674.0,3.0,0.018669,0.186694
455496,Zimbabwe,2024-12-26,0.0,266399,0.0,5740,0.0,0.0,-3.3,-3.4,...,2.154663,0.391389,1675.0,1757.0,1717.0,1757.0,1675.0,3.0,0.018669,0.186694
455497,Zimbabwe,2024-12-27,0.0,266399,0.0,5740,0.0,0.0,-3.3,-3.4,...,2.154663,0.391389,1676.0,1758.0,1718.0,1758.0,1676.0,3.0,0.018669,0.186694
455498,Zimbabwe,2024-12-28,0.0,266399,0.0,5740,0.0,0.0,-3.3,-3.4,...,2.154663,0.391389,1677.0,1759.0,1719.0,1759.0,1677.0,3.0,0.018669,0.186694


CLEANING HOSPITAL

In [157]:
hospital[hospital["daily_occupancy_icu"].isnull()]

,country,country_code,date,daily_occupancy_icu,daily_occupancy_icu_per_1m,daily_occupancy_hosp,daily_occupancy_hosp_per_1m,weekly_admissions_icu,weekly_admissions_icu_per_1m,weekly_admissions_hosp,weekly_admissions_hosp_per_1m
5877,Bulgaria,BGR,2023-01-01,NaN,NaN,351.0,51.755,NaN,NaN,NaN,NaN
8629,Croatia,HRV,2020-04-15,NaN,NaN,341.0,84.608,NaN,NaN,NaN,NaN
8630,Croatia,HRV,2020-04-16,NaN,NaN,372.0,92.299,NaN,NaN,NaN,NaN
8631,Croatia,HRV,2020-04-17,NaN,NaN,372.0,92.299,NaN,NaN,NaN,NaN
8632,Croatia,HRV,2020-04-18,NaN,NaN,361.0,89.570,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
44193,United Kingdom,GBR,2023-07-08,NaN,NaN,1154.0,17.094,NaN,NaN,NaN,NaN
44194,United Kingdom,GBR,2023-07-09,NaN,NaN,1154.0,17.094,NaN,NaN,NaN,NaN
44195,United Kingdom,GBR,2023-07-10,NaN,NaN,1148.0,17.005,NaN,NaN,NaN,NaN
44196,United Kingdom,GBR,2023-07-11,NaN,NaN,1146.0,16.976,NaN,NaN,NaN,NaN


In [158]:
values = {}
for col in hospital.columns:
  if col != "country" and col != "date" and col != "country_code":
    values[col] = hospital[col].median()
hospital = hospital.fillna(value=values)

CLEANING VACCINE_GLOBAL

In [159]:
vaccinations_global

,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations,daily_vaccinations_smoothed,daily_people_vaccinated_smoothed,total_vaccinations_per_hundred,...,people_fully_vaccinated_interpolated,total_boosters_interpolated,total_vaccinations_no_boosters_interpolated,total_vaccinations_no_boosters_per_hundred_interpolated,rolling_vaccinations_6m,rolling_vaccinations_6m_per_hundred,rolling_vaccinations_9m,rolling_vaccinations_9m_per_hundred,rolling_vaccinations_12m,rolling_vaccinations_12m_per_hundred
0,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,...,5.562400e+04,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1366.666667,1366.666667,NaN,...,5.562400e+04,0.000000e+00,1.366667e+03,NaN,1366.666667,0.003368,1.366667e+03,0.003368,1.366667e+03,0.003368
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1366.666667,1366.666667,NaN,...,5.562400e+04,0.000000e+00,2.733333e+03,NaN,2733.333333,0.006736,2.733333e+03,0.006736,2.733333e+03,0.006736
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1366.666667,1366.666667,NaN,...,5.562400e+04,0.000000e+00,4.100000e+03,NaN,4100.000000,0.010104,4.100000e+03,0.010104,4.100000e+03,0.010104
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1366.666667,1366.666667,NaN,...,5.562400e+04,0.000000e+00,5.466667e+03,NaN,5466.666667,0.013472,5.466667e+03,0.013472,5.466667e+03,0.013472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203052,Zimbabwe,2023-12-27,NaN,NaN,NaN,NaN,NaN,3822.227679,2428.736607,NaN,...,5.653986e+06,2.288662e+06,1.163116e+07,NaN,699467.665179,4.352884,1.047290e+06,6.517434,1.395113e+06,8.681983
203053,Zimbabwe,2023-12-28,NaN,NaN,NaN,NaN,NaN,3822.227679,2428.736607,NaN,...,5.656020e+06,2.291488e+06,1.163216e+07,NaN,699467.665179,4.352884,1.047290e+06,6.517434,1.395113e+06,8.681983
203054,Zimbabwe,2023-12-29,NaN,NaN,NaN,NaN,NaN,3822.227679,2428.736607,NaN,...,5.658053e+06,2.294315e+06,1.163315e+07,NaN,699467.665179,4.352884,1.047290e+06,6.517434,1.395113e+06,8.681983
203055,Zimbabwe,2023-12-30,NaN,NaN,NaN,NaN,NaN,3822.227679,2428.736607,NaN,...,5.660086e+06,2.297141e+06,1.163415e+07,NaN,699467.665179,4.352884,1.047290e+06,6.517434,1.395113e+06,8.681983


In [160]:
vaccinations_global.isnull().sum()

,0
country,0
date,0
total_vaccinations,119835
people_vaccinated,124046
people_fully_vaccinated,126046
total_boosters,144273
daily_vaccinations,134181
daily_vaccinations_smoothed,217
daily_people_vaccinated_smoothed,217
total_vaccinations_per_hundred,119835


In [161]:
#There are so many nulls in the data (over 100k per column out of 200k rows), so I decided the best course of action
#is to remove those columns
vaccine_null = vaccinations_global.isnull().sum()
for col in vaccinations_global.columns:
  if vaccine_null[col] > 50000:
    vaccinations_global = vaccinations_global.drop(col, axis=1)
vaccinations_global = vaccinations_global.fillna(0)

In [162]:
vaccinations_global.columns

Index(['country', 'date', 'daily_vaccinations_smoothed',
       'daily_people_vaccinated_smoothed',
       'daily_people_vaccinated_smoothed_per_hundred',
       'daily_vaccinations_smoothed_per_million',
       'total_vaccinations_interpolated', 'people_vaccinated_interpolated',
       'people_fully_vaccinated_interpolated', 'total_boosters_interpolated',
       'total_vaccinations_no_boosters_interpolated',
       'rolling_vaccinations_6m', 'rolling_vaccinations_6m_per_hundred',
       'rolling_vaccinations_9m', 'rolling_vaccinations_9m_per_hundred',
       'rolling_vaccinations_12m', 'rolling_vaccinations_12m_per_hundred'],
      dtype='object')

Cleaning testing


In [163]:
testing

,country,date,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_7day_smoothed,new_tests_per_thousand_7day_smoothed
0,Afghanistan,2022-01-29,853003.0,NaN,21.272,NaN,NaN,NaN
1,Afghanistan,2022-01-30,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2022-01-31,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2022-02-01,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2022-02-02,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
106783,Zimbabwe,2022-06-18,NaN,NaN,NaN,NaN,1782.0,0.111
106784,Zimbabwe,2022-06-19,NaN,NaN,NaN,NaN,1790.0,0.112
106785,Zimbabwe,2022-06-20,2374300.0,NaN,148.454,NaN,1880.0,0.118
106786,Zimbabwe,2022-06-21,NaN,NaN,NaN,NaN,2063.0,0.129


In [164]:
#For the columns total_tests and total_tests_per_thousand, I decided the best way to deal with the nulls
#was to replace the null by the previous value, assuming no new tests were taken, rather than cutting the data or setting it to 0
for country in testing["country"].unique():
  for row in testing[testing["country"] == country]["total_tests_per_thousand"].index:
    if np.isnan(testing.iloc[row, 4]):
      testing.iloc[row, 4] = testing.iloc[row-1, 4]

for country in testing["country"].unique():
  for row in testing[testing["country"] == country]["total_tests"].index:
    if np.isnan(testing.iloc[row, 2]):
      testing.iloc[row, 2] = testing.iloc[row-1, 2]

In [165]:
testing = testing.fillna(0)

In [166]:
testing.isnull().sum()

,0
country,0
date,0
total_tests,0
new_tests,0
total_tests_per_thousand,0
new_tests_per_thousand,0
new_tests_7day_smoothed,0
new_tests_per_thousand_7day_smoothed,0


Cleaning oxcgrt_policy (goverment response)

In [167]:
oxcgrt_policy

,country,date,c1m_school_closing,c2m_workplace_closing,c3m_cancel_public_events,c4m_restrictions_on_gatherings,c5m_close_public_transport,c6m_stay_at_home_requirements,c7m_restrictions_on_internal_movement,c8ev_international_travel_controls,...,v2a_vaccine_availability__summary,v2b_vaccine_age_eligibility_availability_age_floor__general_population_summary,v2c_vaccine_age_eligibility_availability_age_floor__at_risk_summary,stringency_index,containment_health_index,v2_vaccine_availability__summary,v2_pregnant_people,stringency_index_nonvax,stringency_index_vax,stringency_index_weighted_average
0,Afghanistan,2020-01-01,0,0,0,0,0,0,0,0,...,0,NaN,NaN,0.00,0.00,0,NaN,0.00,0.00,0.00
1,Afghanistan,2020-01-02,0,0,0,0,0,0,0,0,...,0,NaN,NaN,0.00,0.00,0,NaN,0.00,0.00,0.00
2,Afghanistan,2020-01-03,0,0,0,0,0,0,0,0,...,0,NaN,NaN,0.00,0.00,0,NaN,0.00,0.00,0.00
3,Afghanistan,2020-01-04,0,0,0,0,0,0,0,0,...,0,NaN,NaN,0.00,0.00,0,NaN,0.00,0.00,0.00
4,Afghanistan,2020-01-05,0,0,0,0,0,0,0,0,...,0,NaN,NaN,0.00,0.00,0,NaN,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202755,Zimbabwe,2022-12-27,1,2,1,0,0,0,0,1,...,3,5-15 yrs,5-15 yrs,29.48,41.65,3,0.0,30.56,26.85,29.48
202756,Zimbabwe,2022-12-28,1,2,1,0,0,0,0,1,...,3,5-15 yrs,5-15 yrs,29.48,41.65,3,0.0,30.56,26.85,29.48
202757,Zimbabwe,2022-12-29,1,2,1,0,0,0,0,1,...,3,5-15 yrs,5-15 yrs,29.48,41.65,3,0.0,30.56,26.85,29.48
202758,Zimbabwe,2022-12-30,1,2,1,0,0,0,0,1,...,3,5-15 yrs,5-15 yrs,29.48,41.65,3,0.0,30.56,26.85,29.48


In [168]:
oxcgrt_policy.isnull().sum()

,0
country,0
date,0
c1m_school_closing,0
c2m_workplace_closing,0
c3m_cancel_public_events,0
c4m_restrictions_on_gatherings,0
c5m_close_public_transport,0
c6m_stay_at_home_requirements,0
c7m_restrictions_on_internal_movement,0
c8ev_international_travel_controls,0


In [169]:
#Dropping unessecary columns
oxcgrt_policy = oxcgrt_policy.drop(["v2b_vaccine_age_eligibility_availability_age_floor__general_population_summary", "v2c_vaccine_age_eligibility_availability_age_floor__at_risk_summary", "v2_pregnant_people"], axis = 1)

In [170]:
#For these columns the best way to handle the nulls are to replace the values of nan with the value before, because the nan meant that
#there wasnt a new value recorded that day
cols = {"e3_fiscal_measures":12, "e4_international_support":13, "h4_emergency_investment_in_healthcare":17, "h5_investment_in_vaccines":18, "stringency_index_weighted_average":27}
for country in oxcgrt_policy["country"].unique():
  for col in cols:
    for row in oxcgrt_policy[oxcgrt_policy["country"] == country][col].index:
      if np.isnan(oxcgrt_policy.iloc[row, cols[col]]):
        try:
          oxcgrt_policy.iloc[row, cols[col]] = oxcgrt_policy.iloc[row-1, cols[col]]
        except:
          oxcgrt_policy.iloc[row, cols[col]] = 0

Cleaning misc data

In [171]:
misc_data

,iso_code,location,date,population,population_density,median_age,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,life_expectancy,human_development_index
0,AFG,Afghanistan,2020-01-05,41128772,54.422,18.6,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,64.83,0.511
1,AFG,Afghanistan,2020-01-06,41128772,54.422,18.6,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,64.83,0.511
2,AFG,Afghanistan,2020-01-07,41128772,54.422,18.6,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,64.83,0.511
3,AFG,Afghanistan,2020-01-08,41128772,54.422,18.6,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,64.83,0.511
4,AFG,Afghanistan,2020-01-09,41128772,54.422,18.6,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429430,ZWE,Zimbabwe,2024-07-31,16320539,42.729,19.6,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,61.49,0.571
429431,ZWE,Zimbabwe,2024-08-01,16320539,42.729,19.6,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,61.49,0.571
429432,ZWE,Zimbabwe,2024-08-02,16320539,42.729,19.6,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,61.49,0.571
429433,ZWE,Zimbabwe,2024-08-03,16320539,42.729,19.6,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,61.49,0.571


In [172]:
misc_data.isnull().sum()

,0
iso_code,0
location,0
date,0
population,0
population_density,68943
median_age,94772
gdp_per_capita,101143
extreme_poverty,217439
cardiovasc_death_rate,100570
diabetes_prevalence,83524


In [173]:
"""
#For these columns the best way to handle the nulls are to replace the values of nan with the value before, because the nan meant that
#there wasnt a new value recorded that day
cols = {"population_density":4, "median_age":5, "gdp_per_capita":6, "extreme_poverty":7, "cardiovasc_death_rate":8, "diabetes_prevalence":9, "female_smokers":10, "male_smokers":11, "handwashing_facilities":12, "life_expectancy": 13, "human_development_index":14}
for country in misc_data["location"].unique():
  for col in cols:
    for row in misc_data[misc_data["location"] == country][col].index:
      if np.isnan(misc_data.iloc[row, cols[col]]):
        try:
          misc_data.iloc[row, cols[col]] = misc_data.iloc[row-1, cols[col]]
        except:
          misc_data.iloc[row, cols[col]] = 0
"""

'\n#For these columns the best way to handle the nulls are to replace the values of nan with the value before, because the nan meant that\n#there wasnt a new value recorded that day\ncols = {"population_density":4, "median_age":5, "gdp_per_capita":6, "extreme_poverty":7, "cardiovasc_death_rate":8, "diabetes_prevalence":9, "female_smokers":10, "male_smokers":11, "handwashing_facilities":12, "life_expectancy": 13, "human_development_index":14}\nfor country in misc_data["location"].unique():\n  for col in cols:\n    for row in misc_data[misc_data["location"] == country][col].index:\n      if np.isnan(misc_data.iloc[row, cols[col]]):\n        try:\n          misc_data.iloc[row, cols[col]] = misc_data.iloc[row-1, cols[col]]\n        except:\n          misc_data.iloc[row, cols[col]] = 0\n'

In [174]:
#misc_data = misc_data.fillna(0)

In [175]:
#misc_data.to_csv("misc_data.csv")

In [176]:
misc_data = pd.read_csv("misc_data.csv")

Creating a relation sql database


In [177]:
conn = sqlite3.connect('covid_data.db')
cursor = conn.cursor()

In [178]:
datasets = {"cases_deaths" :cases_deaths, "hospital": hospital, "vaccinations_global": vaccinations_global, "testing": testing, "xcgrt_policy": oxcgrt_policy, "misc_data":misc_data}

In [179]:
"""
for dataset in datasets:
    datasets[dataset].to_sql(dataset, conn, if_exists='replace', index=False)
"""

In [180]:
hospital.isnull().sum()

,0
country,0
country_code,0
date,0
daily_occupancy_icu,0
daily_occupancy_icu_per_1m,0
daily_occupancy_hosp,0
daily_occupancy_hosp_per_1m,0
weekly_admissions_icu,0
weekly_admissions_icu_per_1m,0
weekly_admissions_hosp,0


In [181]:
pd.read_sql_query("""
SELECT *
FROM cases_deaths
INNER JOIN hospital
ON cases_deaths.country = hospital.country
AND cases_deaths.date = hospital.date
""", conn)

,country,date,new_cases,total_cases,new_deaths,total_deaths,weekly_cases,weekly_deaths,weekly_pct_growth_cases,weekly_pct_growth_deaths,...,country_code,date,daily_occupancy_icu,daily_occupancy_icu_per_1m,daily_occupancy_hosp,daily_occupancy_hosp_per_1m,weekly_admissions_icu,weekly_admissions_icu_per_1m,weekly_admissions_hosp,weekly_admissions_hosp_per_1m
0,Algeria,2020-07-17,585.0,21355,12.0,1040,3547.0,64.0,12.6,6.7,...,DZA,2020-07-17,62.0,1.381,776.0,74.236,92.0,4.645,864.0,56.277
1,Algeria,2020-07-18,593.0,21948,5.0,1045,3706.0,61.0,16.8,3.4,...,DZA,2020-07-18,67.0,1.492,776.0,74.236,92.0,4.645,864.0,56.277
2,Algeria,2020-07-20,535.0,23084,10.0,1066,3889.0,67.0,19.5,13.6,...,DZA,2020-07-20,64.0,1.425,776.0,74.236,92.0,4.645,864.0,56.277
3,Algeria,2020-07-21,607.0,23691,9.0,1075,4002.0,69.0,21.8,16.9,...,DZA,2020-07-21,56.0,1.247,776.0,74.236,92.0,4.645,864.0,56.277
4,Algeria,2020-07-22,587.0,24278,13.0,1088,4062.0,72.0,21.7,20.0,...,DZA,2020-07-22,51.0,1.136,776.0,74.236,92.0,4.645,864.0,56.277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41538,United States,2024-04-21,NaN,103436829,0.0,1186893,NaN,501.0,NaN,-12.4,...,USA,2024-04-21,601.0,1.777,4930.0,14.573,92.0,4.645,5711.0,16.882
41539,United States,2024-04-22,NaN,103436829,0.0,1186893,NaN,501.0,NaN,-12.4,...,USA,2024-04-22,611.0,1.806,5069.0,14.984,92.0,4.645,5658.0,16.725
41540,United States,2024-04-23,NaN,103436829,428.0,1187321,NaN,428.0,NaN,-14.6,...,USA,2024-04-23,605.0,1.788,4878.0,14.420,92.0,4.645,5560.0,16.436
41541,United States,2024-04-24,NaN,103436829,0.0,1187321,NaN,428.0,NaN,-14.6,...,USA,2024-04-24,598.0,1.768,4834.0,14.290,92.0,4.645,5494.0,16.241
